# Djikstra's Algorithm on Bart Station Graph

In [1]:
import csv

import math
import numpy as np
import pandas as pd

import psycopg2

#
# function to run a select query and return rows in a pandas dataframe
# pandas puts all numeric values from postgres to float
# if it will fit in an integer, change it to integer
#

def my_select_query_pandas(query, rollback_before_flag, rollback_after_flag):
    "function to run a select query and return rows in a pandas dataframe"
    
    if rollback_before_flag:
        connection.rollback()
    
    df = pd.read_sql_query(query, connection)
    
    if rollback_after_flag:
        connection.rollback()
    
    # fix the float columns that really should be integers
    
    for column in df:
    
        if df[column].dtype == "float64":

            fraction_flag = False

            for value in df[column].values:
                
                if not np.isnan(value):
                    if value - math.floor(value) != 0:
                        fraction_flag = True

            if not fraction_flag:
                df[column] = df[column].astype('Int64')
    
    return(df)
    

In [2]:
connection = psycopg2.connect(
    user = "postgres",
    password = "ucb",
    host = "postgres",
    port = "5432",
    database = "postgres"
)

In [3]:
cursor = connection.cursor()

In [4]:
def my_read_csv_file(file_name, limit):
    "read the csv file and print only the first limit rows"
    
    csv_file = open(file_name, "r")
    
    csv_data = csv.reader(csv_file)
    
    i = 0
    
    for row in csv_data:
        i += 1
        if i <= limit:
            print(row)
            
    print("\nPrinted ", min(limit, i), "lines of ", i, "total lines.")

In [5]:
import neo4j
driver = neo4j.GraphDatabase.driver(uri="neo4j://neo4j:7687", auth=("neo4j","ucb_mids_w205"))
session = driver.session(database="neo4j")

In [6]:
def my_neo4j_wipe_out_database():
    "wipe out database by deleting all nodes and relationships"
    
    query = "match (node)-[relationship]->() delete node, relationship"
    session.run(query)
    
    query = "match (node) delete node"
    session.run(query)

In [7]:
def my_neo4j_run_query_pandas(query, **kwargs):
    "run a query and return the results in a pandas dataframe"
    
    result = session.run(query, **kwargs)
    
    df = pd.DataFrame([r.values() for r in result], columns=result.keys())
    
    return df

In [8]:
def my_neo4j_number_nodes_relationships():
    "print the number of nodes and relationships"
   
    
    query = """
        match (n) 
        return n.name as node_name, labels(n) as labels
        order by n.name
    """
    
    df = my_neo4j_run_query_pandas(query)
    
    number_nodes = df.shape[0]
    
    
    query = """
        match (n1)-[r]->(n2) 
        return n1.name as node_name_1, labels(n1) as node_1_labels, 
            type(r) as relationship_type, n2.name as node_name_2, labels(n2) as node_2_labels
        order by node_name_1, node_name_2
    """
    
    df = my_neo4j_run_query_pandas(query)
    
    number_relationships = df.shape[0]
    
    print("-------------------------")
    print("  Nodes:", number_nodes)
    print("  Relationships:", number_relationships)
    print("-------------------------")


In [9]:
def my_neo4j_create_node(station_name):
    "create a node with label Station"
    
    query = """
    
    CREATE (:Station {name: $station_name})
    
    """
    
    session.run(query, station_name=station_name)
    

In [10]:
def my_neo4j_create_relationship_one_way(from_station, to_station, weight):
    "create a relationship one way between two stations with a weight"
    
    query = """
    
    MATCH (from:Station), 
          (to:Station)
    WHERE from.name = $from_station and to.name = $to_station
    CREATE (from)-[:LINK {weight: $weight}]->(to)
    
    """
    
    session.run(query, from_station=from_station, to_station=to_station, weight=weight)
    

In [11]:
def my_neo4j_create_relationship_two_way(from_station, to_station, weight):
    "create relationships two way between two stations with a weight"
    
    query = """
    
    MATCH (from:Station), 
          (to:Station)
    WHERE from.name = $from_station and to.name = $to_station
    CREATE (from)-[:LINK {weight: $weight}]->(to),
           (to)-[:LINK {weight: $weight}]->(from)
    
    """
    
    session.run(query, from_station=from_station, to_station=to_station, weight=weight)
    

In [12]:
connection = psycopg2.connect(
    user = "postgres",
    password = "ucb",
    host = "postgres",
    port = "5432",
    database = "postgres"
)

In [13]:
cursor = connection.cursor()

In [14]:
my_neo4j_wipe_out_database()

In [15]:
my_neo4j_number_nodes_relationships()

-------------------------
  Nodes: 0
  Relationships: 0
-------------------------


In [16]:
connection.rollback()

query = """

select station
from stations
order by station

"""

cursor.execute(query)

connection.rollback()

rows = cursor.fetchall()

print(rows)

for row in rows:
    
    station = row[0]
    
    my_neo4j_create_node('depart ' + station)
    my_neo4j_create_node('arrive ' + station)
    

[('12th Street',), ('16th Street Mission',), ('19th Street',), ('24th Street Mission',), ('Antioch',), ('Ashby',), ('Balboa Park',), ('Bay Fair',), ('Berryessa',), ('Castro Valley',), ('Civic Center',), ('Coliseum',), ('Colma',), ('Concord',), ('Daly City',), ('Downtown Berkeley',), ('Dublin',), ('El Cerrito del Norte',), ('El Cerrito Plaza',), ('Embarcadero',), ('Fremont',), ('Fruitvale',), ('Glen Park',), ('Hayward',), ('Lafayette',), ('Lake Merritt',), ('MacArthur',), ('Millbrae',), ('Milpitas',), ('Montgomery Street',), ('North Berkeley',), ('North Concord',), ('OAK',), ('Orinda',), ('Pittsburg',), ('Pittsburg Center',), ('Pleasant Hill',), ('Powell Street',), ('Richmond',), ('Rockridge',), ('San Bruno',), ('San Leandro',), ('SFO',), ('South Hayward',), ('South San Francisco',), ('Union City',), ('Walnut Creek',), ('Warm Springs',), ('West Dublin',), ('West Oakland',)]


In [17]:
connection.rollback()

query = """

select station, line
from lines
order by station, line

"""

cursor.execute(query)

connection.rollback()

rows = cursor.fetchall()

print(rows)

[('12th Street', 'orange'), ('12th Street', 'red'), ('12th Street', 'yellow'), ('16th Street Mission', 'blue'), ('16th Street Mission', 'green'), ('16th Street Mission', 'red'), ('16th Street Mission', 'yellow'), ('19th Street', 'orange'), ('19th Street', 'red'), ('19th Street', 'yellow'), ('24th Street Mission', 'blue'), ('24th Street Mission', 'green'), ('24th Street Mission', 'red'), ('24th Street Mission', 'yellow'), ('Antioch', 'yellow'), ('Ashby', 'orange'), ('Ashby', 'red'), ('Balboa Park', 'blue'), ('Balboa Park', 'green'), ('Balboa Park', 'red'), ('Balboa Park', 'yellow'), ('Bay Fair', 'blue'), ('Bay Fair', 'green'), ('Bay Fair', 'orange'), ('Berryessa', 'green'), ('Berryessa', 'orange'), ('Castro Valley', 'blue'), ('Civic Center', 'blue'), ('Civic Center', 'green'), ('Civic Center', 'red'), ('Civic Center', 'yellow'), ('Coliseum', 'blue'), ('Coliseum', 'gray'), ('Coliseum', 'green'), ('Coliseum', 'orange'), ('Colma', 'red'), ('Colma', 'yellow'), ('Concord', 'yellow'), ('Daly 

In [18]:
for row in rows:
    
    station = row[0]
    line = row[1]
    
    my_neo4j_create_node(line + ' ' + station)
    my_neo4j_create_relationship_one_way('depart ' + station, line + ' ' + station,0)
    my_neo4j_create_relationship_one_way(line + ' ' + station, 'arrive ' + station,0)
    
    

In [19]:
my_neo4j_number_nodes_relationships()

-------------------------
  Nodes: 214
  Relationships: 228
-------------------------


In [20]:
connection.rollback()

query = """

select a.station, a.line as from_line, b.line as to_line, s.transfer_time
from lines a
     join lines b
       on a.station = b.station and a.line <> b.line 
     join stations s
       on a.station = s.station
order by 1, 2, 3

"""
cursor.execute(query)

connection.rollback()

rows = cursor.fetchall()

for row in rows:
    
    station = row[0]
    from_line = row[1]
    to_line = row[2]
    transfer_time = int(row[3])
    
    
    my_neo4j_create_relationship_one_way(from_line + ' ' + station, to_line + ' ' + station, transfer_time)
    


In [21]:
my_neo4j_number_nodes_relationships()

-------------------------
  Nodes: 214
  Relationships: 436
-------------------------


In [22]:
connection.rollback()

query = """

select a.line, a.station as from_station, b.station as to_station, t.travel_time
from lines a
  join lines b
    on a.line = b.line and b.sequence = (a.sequence + 1)
  join travel_times t
    on (a.station = t.station_1 and b.station = t.station_2)
        or (a.station = t.station_2 and b.station = t.station_1)
order by line, from_station, to_station

"""

cursor.execute(query)

connection.rollback()

rows = cursor.fetchall()

In [23]:
for row in rows:
    
    line = row[0]
    from_station = row[1]
    to_station = row[2]
    travel_time = int(row[3])
    
    my_neo4j_create_relationship_two_way(line + ' ' + from_station, line + ' ' + to_station, travel_time)
    

In [24]:
my_neo4j_number_nodes_relationships()

-------------------------
  Nodes: 214
  Relationships: 652
-------------------------


In [25]:
query = "CALL gds.graph.drop('ds_graph', false)"
session.run(query)

query = """

CALL gds.graph.project('ds_graph', 'Station', 'LINK', 
                      {relationshipProperties: 'weight'})
"""

session.run(query)

In [26]:
def djikstra(source, target):
    query = """
    MATCH (source:Station {name: $sourceName}), (target:Station {name: $targetName})
    CALL gds.allShortestPaths.dijkstra.stream('ds_graph', {
        sourceNode: source,
        relationshipWeightProperty: 'weight'
    })
    YIELD index, sourceNode, targetNode, totalCost, nodeIds, costs, path WHERE target IN gds.util.asNodes(nodeIds)
    RETURN
        index,
        gds.util.asNode(sourceNode).name AS sourceNodeName,
        gds.util.asNode(targetNode).name AS targetNodeName,
        totalCost,
        [nodeId IN nodeIds | gds.util.asNode(nodeId).name] AS nodeNames,
        costs,
        nodes(path) as path
    ORDER BY index
    """
    temp_df = my_neo4j_run_query_pandas(query, sourceName = "depart " + source, targetName = "arrive " + target)
    session.run(query, sourceName = "depart " + source, targetName = "arrive " + target)
    return temp_df

In [27]:
def format_djikstra(djikstra_df):
    temp1 = djikstra_df['nodeNames'].explode().reset_index()
    temp2 = djikstra_df['costs'].explode().reset_index()
    return pd.concat([temp1,temp2], axis=1)[['nodeNames','costs']].rename(columns={"nodeNames":"Station name", "costs": "Total travel time"})
    

In [28]:
nb_wd_df = djikstra("North Berkeley", "West Dublin")
results = format_djikstra(nb_wd_df)

In [29]:
results

,Station name,Total travel time
0,depart North Berkeley,0.0
1,orange North Berkeley,0.0
2,orange Downtown Berkeley,120.0
3,orange Ashby,300.0
4,orange MacArthur,540.0
5,orange 19th Street,720.0
6,orange 12th Street,840.0
7,orange Lake Merritt,1020.0
8,orange Fruitvale,1320.0
9,orange Coliseum,1560.0


In [30]:
shortest_path_df = nb_wd_df.rename(columns = {"totalCost": "travelTime", "nodeNames": "pathTaken"})[['sourceNodeName','targetNodeName','travelTime','pathTaken','costs']]

In [31]:
shortest_path_df

,sourceNodeName,targetNodeName,travelTime,pathTaken,costs
0,depart North Berkeley,arrive West Dublin,2934.0,"[depart North Berkeley, orange North Berkeley,...","[0.0, 0.0, 120.0, 300.0, 540.0, 720.0, 840.0, ..."
